<a href="https://colab.research.google.com/github/nnilayy/Unet/blob/main/Polygon2Mask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
import numpy as np
from skimage import draw

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [21]:
!unzip -q /content/Images.zip -d /content/

## Extracting the shape of the images

In [ ]:
from google.colab.patches import cv2_imshow
import glob
import os
import numpy as np
import cv2

images=sorted(glob.glob("/content/Images/*"))
image_shapes=[]
for image in images:
  image=cv2.imread(image)
  image=np.array(image)
  width,height,_=image.shape
  cv2_imshow(image)
  image_shapes.append((width,height))
image_shapes


## Extracting the X_coordinates and y_coordinates of all images 

In [ ]:
import json
with open('/content/Balls_Polygon.json', 'r') as f:
  file = json.load(f)
x=[]
y=[]
for i in file:
 x.append(file[str(i)]["regions"]["0"]["shape_attributes"]['all_points_x'])
 y.append(file[str(i)]["regions"]["0"]["shape_attributes"]['all_points_y'])
print(x)
print(y)

## Drawing masks

In [42]:
def mask(x, y, shape):
    row_fill, col_fill = draw.polygon(x, y, shape)
    mask = np.zeros(shape, dtype=np.bool)
    mask[row_fill,col_fill] = True
    return mask*255

In [44]:
!mkdir "/content/drive/MyDrive/Bitmaps"

In [ ]:
from google.colab.patches import cv2_imshow
import cv2
import os
img=[]
for i,shape in zip(range(len(x)),image_shapes):
  bw=mask(y[i],x[i],shape)
  img.append(bw)
  cv2.imwrite("/content/drive/MyDrive/Bitmaps/"+str(i+1)+".jpeg",bw)
  cv2_imshow(bw)

## Setting Data Generator For Augmentation

In [36]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
datagen=ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest",
)

## Augmenting Images

In [ ]:
!mkdir "/content/drive/MyDrive/Augmented"

In [ ]:
images=sorted(glob.glob("/content/Images/*"))
masks=sorted(glob.glob("/content/drive/MyDrive/Bitmaps*"))

for image,mask in zip(images,masks):
  img=load_img(image)
  x=img_to_array(img)
  x=x.reshape((1,)+x.shape)

  mask=load_img(mask)
  y=img_to_array(mask)
  y=y.reshape((1,)+y.shape)

  i=1
  for batch in datagen.flow(x,batch_size=1,save_to_dir="/content/drive/MyDrive/Augmented/Images/",save_prefix="Balls",save_format="jpeg",seed=42):
    i+=1
    if i>5:
      break
  i=1
  for batch in datagen.flow(y,batch_size=1,save_to_dir="/content/drive/MyDrive/Augmented/Masks/",save_prefix="Balls",save_format="jpeg",seed=42):
    i+=5
    if i>3:
      break



In [ ]:
# image=[]
# for i in range(len(x)):
#   image.append([])
#   for j in range(len(x[i])):
#     image[i].append((x[i][j],y[i][j]))
# image
# for a in range(len(x)):

# x_coordinates=[]
# for i in range(len(x)):
#   for j in range(len(x[i])):
#     x_coordinates.append(x[i][j])
# print(len(x_coordinates))

# y_coordinates=[]
# for i in range(len(x)):
#   for j in range(len(x[i])):
#     y_coordinates.append(x[i][j])
# print(len(y_coordinates))


# x_coordinates=[]
# y_coordinates=[]
# polygon=[]
# for i in zip(range(len(x))):
#   for j in zip(range(len(x[i]))):
#     x_coordinates.append(x[i][j])
#     y_coordinates.append(y[i][j])

# for x,y in zip(x_coordinates,y_coordinates):
#   c=(x,y)
#   polygon.append(c)
# polygon

# a=file["1.jpeg"]["regions"]["0"]["shape_attributes"]['all_points_x']
# b=file["1.jpeg"]["regions"]["0"]["shape_attributes"]['all_points_y']
# print(a)
# print(b)

# x=sum(x, [])
# y=sum(y, [])

In [ ]:
# if the opening and closing brackets are {} we can use string indexes, otherwise if we the opening and closing brackets are [], we use integer index

In [ ]:
# import json
# person = '{"name": "Bob", "languages": ["Python", "Java","Loda","Lehsun"]}'

# person = '{"name": "Bob", "languages": ["English", "French"]}'
# person_dict = json.loads(person)

# Output: {'name': 'Bob', 'languages': ['English', 'French']}
# print( person_dict)

# Output: ['English', 'French']
# print(person_dict['languages'])